# Driver Drowsiness Detection System using Neural Network

Step 1: Install Required Libraries
You'll need to install the following libraries:

OpenCV: for capturing video from the webcam.
TensorFlow/Keras: for building and training the neural network.
Streamlit: for creating the GUI.
Scipy: for playing the buzzer sound.
dlib: for facial landmark detection.

Step 2: Capture Video from Webcam
Create a function to capture video from the webcam and analyze each frame.

In [1]:
import cv2

def capture_video():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Process the frame here
        cv2.imshow('Driver Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


Step 3: Build the Neural Network
Create a neural network model to detect drowsiness using TensorFlow/Keras.

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


Step 4: Detect Drowsiness Using Facial Landmarks
Use dlib to detect facial landmarks and extract the eye region to determine if the eyes are closed.

In [3]:
import dlib
from scipy.spatial import distance

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def get_eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_drowsiness(frame, gray):
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        left_eye = [landmarks.part(i) for i in range(36, 42)]
        right_eye = [landmarks.part(i) for i in range(42, 48)]
        left_ear = get_eye_aspect_ratio(left_eye)
        right_ear = get_eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0
        if ear < 0.25:
            return True
    return False
